In [ ]:
import pandas as pd
import datetime

In [ ]:
def convert_date(df):
    df['Corrected_Dates'] = pd.to_datetime((df['Date'] - 693963), unit='D', origin=datetime.datetime(1900, 1, 1))
    return df

In [ ]:
def show_max_min_dates(df):
    max_date = df['Corrected_Dates'].max()
    min_date = df['Corrected_Dates'].min()
    print("Maximum Corrected Date: ", max_date)
    print("Minimum Corrected Date: ", min_date)

In [ ]:
df = pd.read_csv('RRCA_baseflow.csv')
df = convert_date(df)
show_max_min_dates(df)
df